# Container Pre-marshalling Problem Enviroment
En el siguiente documento se documentará cómo es que funciona el CPMP Enviroment

## Contenidos
[Valores Relevantes](#relevant)

[Acciones](#actions)

[Observaciones](#observation)

[Demonstración](#demo)
- [Crear el estado y Manipularlo](#createstate)
- [Representación del Patio de Containers (yard.py)](#representation)
- [Ambiente de CPMP (containeryard.py)](#ambient)
- [Pruebas](#test)

[Generador de Instancias](#generate)

[Ejecutar el Ambiente](#execute)



## <a name="relevant"></a> Valores Relevantes

state : Es la representación del estado actual, de tipo 'Yard' para permitir realizarle cambios al estado

showDebug : Valor booleano que dicta si mostrará información de Debug o no

max_step : Pasos máximos, cálculados por una solución greedy. La idea es que la red mejore lo que dicta el greedy en StackedYard.py

training :  booleano que dice si está entrenando o no. Más que nada para ver qué conjunto utilizará para trabajar.

fileStack :  Los archivos que contienen los estados

current_step : Paso actual del estado, la idea es que sea menor a max_step

last_reward : la última recompenza obtenida, utilizada para recompenzar la diferencia entre la actual recompenza y la anterior (last_reward)


## <a name="actions"></a>Acciones
La acción que entregará como output va a ser un número de 0 a x, dónde x será la anchura del estado

Este valor representa el stack en el cuál se debe realizar la acción. En la actualidad, la red identificará un stack y una función de greedy realizará el cambio en dicho stack seleccionado.

```py
self.action_space = spaces.Discrete(x) # de 0 a x
```

## <a name="observation"></a>Observaciones
La observación será un arreglo que contendrá:

1)  El estado actual del patio con los containers, primero los datos vacios y luego los que poseen un valor, de tal manera que los que tienen valor se "eleven" y dejen a los ceros arriba. Posterior a esto, se realizaran dos normalizaciones, una que "Compactará" los valores, para que siempre esten en rangos iguales, y, una normalización de mayor y menor. Algo así:

![Imagen Explicativa de Observacion](https://i.imgur.com/0u1Exr9.png "Ejemplo Observacion")

Los valores equivalentes al container son numeros enteros pertenecientes a `[0,∞)`. 
La representación dentro de la observación los normalizara en un rango de `[0,1]`, utilizando una normalización min-max, dónde el minimo y el maximo dependera del  problema que se este resolviendo.

2) Además de los valores del estado en sí, se cuenta con un valor booleano (0 ó 1) que representará si una columna se encuentra en orden (1) o no (0).


Algo así se vería lo que alimenta a la red.
```
[0.00833333 0.06666667 0.05       0.01666667 0.025      0.06666667
 0.06666667 0.06666667 0.06666667 0.01666667 0.05833333 0.06666667
 0.06666667 0.00833333 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.04166667 0.03333333 0.06666667 0.06666667 0.05833333
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.05833333
 0.05833333 0.06666667 0.06666667 0.         0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.03333333 0.03333333 0.06666667
 0.06666667 0.05       0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.05       0.         1.         1.         0.
 1.         1.         1.         1.         1.         0.        ]
```

## <a name="demo"></a>Demonstración
A continuación se explicarán funciones del ambiente con ejemplos, de modo que se pueda comprender mejor cómo es que opera.

Las funciones tendrán los mismos nombres de los métodos encontrados en `containeryard.py` y `yard.py`.

### <a name="createstate"></a>Crear el estado y manipularlo
Para crear el estado, se utilizará `yard.py`, debido a que `containeryard.py` implementará dicha representación a un formato de gym para facilitar el uso en redes de aprendizaje por reforzamiento.

A continuación un ejemplo de cómo se crea el estado, con un ejemplo encontrado en este mismo directorio, `example_10_5_0_226.bay`.

### <a name="representation"></a>Representación del Patio de Containers (yard.py)
Este archivo será la capa que modificará directamente el estado, el ambiente de gym (`containeryard.py`) la utilizará para no trabajarlo directamente.

Los métodos que explicaremos son:

`isStackEmpty(i)`: Retornará si el stack i (de 0 a n stacks) se encuentra vacío o no (True/False). Este método será utilizado para saber si es posible sacar un container de dicho stack, ya que si se encuentra vacío no se podrá sacar nada.

`isStackFull(i)`: Retornará si el stack i (de 0 a n stacks) se encuentra lleno o no (True/False). Este método será utilizado para saber si es posible poner un container en dicho stack, ya que si se encuentra lleno, no se podrá poner nada sobre este.

`moveStack(src, dest)`: Este método utilizará los dos anteriores para ver si es posible realizar una acción. En caso de ser posible, realizará dicha acción y cambiará el estado.

`getAsObservation()`: Este método devolverá el estado en formato de observación (sin la normalización), es decir, entregará el estado con los containers NO vacíos primero, y luego pondrá los vacíos, por ejemplo quedaría así: `[4, 3, 6, 7, 4, 4, 0, 0, 0, 0, 0 , ....]`

`render(self)`: Este método mostrará el estado de una manera "humanamente-leíble"

`isSorted(i)`: Verá si un stack i está ordenado o no.


In [6]:
import numpy as np
import random as rand
import os
import math
import numpy as np

from containeryard.StackedYard import Layout, greedy_solve, read_file, select_destination_stack
from Constants import FILE_PATH

import gym
from gym import error, spaces, utils
from gym.utils import seeding

#Archivo de Prueba
TEST_FILE = r'./example_10_5_0_226.bay'

############################################################################################    
# From yard.py
class Yard():
    x:int
    y:int

    max:int
    min:int

    numTranslation:dict

    def __init__(self, yard):
        #Gets file yard size from file name.
        self.x, self.y = yard.shape # Should Be Numpy
        self.state = yard

        #######END#####################

        self.max = np.amax(self.state)
        self.min = np.amin(self.state)

        #Creates the dictionary to translate to smaller numbers.
        allNumbers = np.unique(self.state)
        self.numTranslation = dict()
        for i in range(allNumbers.size):
            #The value, but normalized.
            if allNumbers[i] == 0:
                #Zero will be one.
                self.numTranslation[allNumbers[i]] = 1.0
            else:
                self.numTranslation[allNumbers[i]] = ((i-1)/((allNumbers.size-1)-1)) 
        ####

    def isStackEmpty(self, i):
        return self.state[i][0] == 0

    def isStackFull(self, i):
        return self.state[i][self.y-1] != 0

    def moveStack(self, src, dest):
        value = 0
        
        #---> Primero, verificamos que la accion se pueda realizar.
        if self.isStackFull(dest) or self.isStackEmpty(src):
            return False

        #---> Segundo, conseguimos y eliminamos el valor en top.
        for pos in range(self.y-1, -1, -1):
            if self.state[src][pos] > 0:
                value = self.state[src][pos]
                self.state[src][pos] = 0
                break

        #---> Dejamos el valor 
        for pos in range(self.y):
            if self.state[dest][pos] == 0:
                self.state[dest][pos] = value
                break
        return True

    def getAsObservation(self):
        stateCopy = np.array(self.state, copy=True, dtype=np.float)
        obs = np.zeros(stateCopy.shape)

        x,y = stateCopy.shape
        for i in range(x):
            parcialObs = np.concatenate((
                stateCopy[i][np.nonzero(stateCopy[i]==0)], ## Dejamos los ceros abajo
                stateCopy[i][np.nonzero(stateCopy[i])] ## y arriba los numeros.
                ))

            obs[i] = np.array(parcialObs, copy=True, dtype=np.float)

        obs = np.rot90(obs)
        obs = np.asarray(obs).ravel()

        for i in range(obs.size):
            obs[i] = self.numTranslation[obs[i]]

        return obs

    def render(self):
        rend = np.rot90(self.state, k=1)
        print(rend)

    def isSorted(self, i):
        lastNum = 999999
        for num in np.array(self.state[i]):
            if num == 0:
                break
            if num > lastNum:
                return False
            lastNum = num
        return True

    def asLayout(self):
        layoutState = []
        for stack in self.state:
            s = stack[np.nonzero(stack)] # Get the non zero values
            if s.size <= 0:
                s = np.array([0])
            layoutState.append(s.tolist())

        return layoutState

    def isDone(self):
        for sort in self.getAllSorts():
            if not sort:
                return False
        return True

    def getAllSorts(self):
        sorted = np.zeros(self.x, dtype=np.bool)
        for i in range(self.x):
            sorted[i] = self.isSorted(i)
        return sorted


### <a name="ambient"></a>Ambiente de CPMP (containeryard.py)
ContainerYard se utilizará para que la red pueda tomar la observación y realizar acciones en el estado. Más información sobre gym puede ser leída en su página web: https://gym.openai.com/

Los métodos que utilizaremos para explicar serán:
`reset()`: Reiniciará el problema actual, cargando otro (En el caso de este ejemplo, siempre cargará uno, pero en la realidad cargará el siguiente!!)

`step(action)`: Realizará una acción utilizando _take_action(action), después, calculará la recompensa del estado y entregará la **observación**, **recompensa**, un **boolean** definiendo si el estado es terminal y, opcionalmente, **información de debug**. 

`_take_action(action)`: Este método realizará la acción en el estado. Primero calculará lo que se demora el greedy en realizar la acción, y lo guardará para calcular la recompensa. Después, utilizará el método de `Yard`, `moveStack` para realizar la acción en el estado.

`_next_observation()`: Entregará la observación para la red, descrita anteriormente en el inicio de este archivo

`render()`: Este método imprimirá en pantalla el estado actual, de modo que una persona pueda leer en lo que va el estado actualmente. En una versión futura se podrá habilitar para que esta visualización sea gráfica en vez de basada en texto en la terminal.


In [10]:
def RandomMovementGeneration(x=20, y=5, difficulty=0): # 0 is easy, 1 is normal, 1 is hard. The more difficulty, the more  movements.
    max_containers = (x*(y-2)) #60
    max_priority=20

    # This generator starts from a solved one and makes random movements.
    yard = np.zeros(shape=(x,y))

    count = 0
    for i in range(x):
        
        stack = []
        height = min(rand.randint(0,y),max_containers-count)
        
        for j in range(height):
            num = rand.randint(1,max_priority)
            stack.append(num)
            count +=1
            
        stack.sort(reverse=True)
        
        j=0
        for s in stack:
            yard[i][j] = s
            j +=1

    state = Yard(yard)
    # How Many Movements
    if difficulty == 0:
        min_moves = 3
        max_moves = 6
    elif difficulty == 1:
        min_moves = 6
        max_moves = 10
    elif difficulty == 2:
        min_moves = 10
        max_moves = 16
    else:
        min_moves = 16
        max_moves = 23

    moves = rand.randint(min_moves ,max_moves)

    oldSet = [-1,-1]
    #for i in range(moves):
    #while np.count_nonzero(state.getAllSorts() == False) <= moves:
    #una vez que un stack recibe un elemento queda bloqueada
    blocked_stacks = []
    
    done = 0
    
    while done < moves:
        a = rand.randint(0,x-1)
        while state.isStackEmpty(a) or a == oldSet[0]:
            a = rand.randint(0,x-1)

        b = rand.randint(0,x-1)
        while state.isStackFull(b) or a==b or b == oldSet[1]:
            b = rand.randint(0,x-1) 
            
        #print(a,b)
        ret = state.moveStack(a,b)
        oldSet = [a,b]
        done += 1
    
    newState = np.array(state.state, copy=True)
    np.random.shuffle(newState)
    state = Yard(newState)

    layoutState = state.asLayout()
    layout = Layout(layoutState, state.y)
    max_step = greedy_solve(layout)

    return state, layout, max_step 


############################################################################################    
# From containeryard.py
class ContainerYard(gym.Env):
    metadata = {'render.modes':['human']}

    state : Yard
    showDebug : bool
    max_stel : int
    training : bool
    fileStack : list
    current_step : int
    last_reward : int

    def __init__(self, showDebug = False, x=20, y=5):
        super(ContainerYard, self).__init__()

        self.x = x
        self.y = y

        ### START OF CONFIG ###
        self.showDebug = showDebug
        self.max_step = 10

        #---> Creting the stack for files to use..
        self.fileStack = []
        
        ############################
        self.current_step = 0
        self.last_reward = 0

        # Start The Episode
        self.reset()

        """ Action Space: Valores del tamano
                vertical del patio.

        """

        self.action_space = spaces.Discrete(self.state.x)

        self.observation_space = spaces.Box(low=-1, high=255, shape=(self.state.x*self.state.y + self.state.x ,), dtype=np.float_)

    def reset(self):
        #Resetting
        self.current_step = 0
        self.state, self.layout, self.max_step = RandomMovementGeneration(x=self.x, y=self.y, difficulty=4)

        return self._next_observation()

    def step(self, action):
        #Taking Action!
        ret = self._take_action(action)

        #New Greedy Value.
        self.greedy_steps = greedy_solve(
            Layout(self.state.asLayout(), self.state.y)
        )

        self.current_step += 1


        formula_reward = np.exp(-(self.current_step + self.greedy_steps))

        reward = formula_reward - self.last_reward

        self.last_reward = formula_reward

        done = (self.state.isDone() or self.current_step >= self.max_step)

        if ret is False:
            #Could not make action, so we punish it.
            reward = -1

        obs = self._next_observation()

        return obs, reward, done, _

    def _take_action(self, action):

        layoutState = self.state.asLayout()
        dest = select_destination_stack(
            Layout(layoutState, self.state.y), 
            action
        )
        return self.state.moveStack(action, dest)

    def _next_observation(self):
        
        #Normalization and Generating the Yard Observation
        obs = self.state.getAsObservation()
        
        #Misc Observation
        for i in range(self.state.x):
            obs = np.insert(obs, obs.size, 1 if self.state.isSorted(i) is True else 0)
        
        #self.state.render()
        return obs

### <a name="test"></a>Pruebas
Aquí realizaremos pruebas para que se comprenda el funcionamiento.

Iniciaremos el estado y probaremos sus funcionamientos básicos.

`gym_representation` será nuestro ambiente de gym, primero mostraremos con `.state.render()` cómo es que verá la persona el ambiente y con `._next_observation()` lo que ve la red.

In [11]:
gym_representation = ContainerYard()

print('PARA HUMANOS')
gym_representation.state.render()

print('Estado transformado: (Como arreglo)')
print(gym_representation.state.getAsObservation())

print('\nObservacion (anterior + 0/1 si stack esta ordenado o no)')
print(gym_representation._next_observation())

PARA HUMANOS
[[ 0.  0.  2.  0. 15.  0.  0.  0.  0.  3.  0.  4.  0.  0.  0. 10.  0.  2.
   0.  0.]
 [ 0.  0.  4.  0.  3.  0.  0.  0.  0.  8.  0.  4.  0.  0.  0. 10.  0. 11.
   8.  0.]
 [ 5.  0.  6.  0.  6.  0.  4.  0.  0. 10.  7.  7.  0.  0.  0. 10.  0. 12.
  10.  6.]
 [16.  0.  8. 15. 12.  0.  6.  0.  3. 17. 14. 20.  0.  0.  0. 16.  0. 14.
  12.  6.]
 [ 1.  2. 20. 13. 17.  0. 16. 19.  2. 20. 20. 20.  0. 11.  0. 18.  0. 19.
  20. 17.]]
Estado transformado: (Como arreglo)
[0.22222222 0.05555556 0.05555556 0.72222222 0.72222222 1.
 0.16666667 0.94444444 0.11111111 0.11111111 0.33333333 0.16666667
 1.         0.5        1.         0.44444444 1.         0.05555556
 0.38888889 0.27777778 0.77777778 1.         0.16666667 0.61111111
 0.11111111 1.         0.27777778 1.         0.05555556 0.38888889
 0.66666667 0.16666667 1.         1.         1.         0.44444444
 1.         0.5        0.44444444 0.27777778 0.         1.
 0.27777778 1.         0.27777778 1.         0.77777778 1.
 1.         0

Ahora, probaremos un par de funciones.
Comenzaremos probando realizar un movimiento sobre la columna numero 3 (Si contamos desde 0, sería la número 2.)

El método retornara cuatro valores:
La nueva observación, la recompensa, si el estado es terminal y, por último, información adicional opcional.

In [12]:
print("Información de Retorno: (Observacion, recompensa, es estado terminal?, información debug [OPCIONAL])")
print(gym_representation.step(2))

print("\nRender:")
gym_representation.state.render()

Información de Retorno: (Observacion, recompensa, es estado terminal?, información debug [OPCIONAL])
(array([0.22222222, 0.05555556, 0.16666667, 0.72222222, 0.72222222,
       1.        , 0.16666667, 0.94444444, 0.11111111, 0.11111111,
       0.33333333, 0.16666667, 1.        , 0.5       , 1.        ,
       0.44444444, 1.        , 0.05555556, 0.38888889, 0.27777778,
       0.77777778, 0.05555556, 0.27777778, 0.61111111, 0.11111111,
       1.        , 0.27777778, 1.        , 0.05555556, 0.38888889,
       0.66666667, 0.16666667, 1.        , 1.        , 1.        ,
       0.44444444, 1.        , 0.5       , 0.44444444, 0.27777778,
       0.        , 1.        , 0.38888889, 1.        , 0.27777778,
       1.        , 0.77777778, 1.        , 1.        , 0.44444444,
       1.        , 0.33333333, 1.        , 1.        , 1.        ,
       0.44444444, 1.        , 0.55555556, 0.55555556, 0.83333333,
       1.        , 1.        , 1.        , 1.        , 0.55555556,
       1.        , 1.      

A continuación utilizaremos los métodos que se utilizan dentro de todas las funciones:

In [13]:
print("Stacks Ordenados: ")
print(gym_representation.state.getAllSorts())


print("\nEs Estado Terminal: ")
print(gym_representation.state.isDone())

print("\nValor Greedy Estado Actual - Pasos Tomados: ")
print(greedy_solve(Layout(gym_representation.state.asLayout(), gym_representation.state.y)), gym_representation.current_step)

print('\nEstado Actual:')
gym_representation.state.render()

Stacks Ordenados: 
[False  True  True False False  True  True  True False  True  True  True
  True  True  True  True  True  True  True  True]

Es Estado Terminal: 
False

Valor Greedy Estado Actual - Pasos Tomados: 
5 1

Estado Actual:
[[ 0.  0.  0.  0. 15.  0.  0.  0.  0.  3.  0.  4.  0.  0.  0. 10.  0.  2.
   0.  0.]
 [ 0.  0.  4.  0.  3.  0.  0.  0.  0.  8.  0.  4.  0.  0.  0. 10.  0. 11.
   8.  0.]
 [ 5.  0.  6.  0.  6.  0.  4.  0.  0. 10.  7.  7.  0.  0.  0. 10.  0. 12.
  10.  6.]
 [16.  2.  8. 15. 12.  0.  6.  0.  3. 17. 14. 20.  0.  0.  0. 16.  0. 14.
  12.  6.]
 [ 1.  2. 20. 13. 17.  0. 16. 19.  2. 20. 20. 20.  0. 11.  0. 18.  0. 19.
  20. 17.]]


## <a name="generator"></a>Generador de Instancias (DEPRECADO)
** Atencion **: El generador esta incluido en la función reset y ya no se utiliza el generador de instancias.

El archivo de generador de instancias se encuentra en la carpeta [instance_generator](https://github.com/Nyuku/CPMP-AI/tree/master/instance_generator) dentro del repositorio.

Alli se podra encontrar el archivo [GenerateRandom.py](https://github.com/Nyuku/CPMP-AI/tree/master/instance_generator/GenerateRandom.py) que generara instancias con valores aleatorios, preocupandose que sea posible de ser resueltos; [ProblemGenerator.py](https://github.com/Nyuku/CPMP-AI/tree/master/instance_generator/ProblemGenerator.py) generara problemas comenzando por uno resuelto, de tal manera que al resuelto se le realizaran movimientos aleatorios para desordenarlo. Por ultimo, [Yard.py](https://github.com/Nyuku/CPMP-AI/tree/master/instance_generator/Yard.py) contiene la estructura del problema, facilitando los movimientos y la insercion. 

### Configurar el Generador
En ambos generadores se encontrara una variable llamada `rootFolder`. En esta variable se tendra que escribir la direccion de la carpeta en la cual se guardaran los datos de prueba y entrenamiento. Es importante que las carpetas `testing` y `training` se encuentren creadas en dicho fichero.

### Ejecutar el Generador
El generador se ejecutara de la siguiente forma, ambos se utilizaran igual.

`python <NOMBRE_ARCHIVO>.py X Y`

Donde X sera el ancho del patio de containers e Y sera el alto. Por ejemplo, si corremos:

`python GenerateRandom.py 20 5`

Este como resultado daria algo asi:

```
Container Size (x,y)
n n n n n n n 
n n n n n n n
n n n n n n n
n n n n n n n 
x = 7; y = 4
```

## <a name="execute"></a>Ejecutar el Ambiente
Un ejemplo de la ejecucion del ambiente puede ser vista en el archivo [DQN_Optimizer.py](https://github.com/Nyuku/CPMP-AI/tree/master/DQN_Optimizer.py). En esta demonstracion se utilizara la libreria `stable_baselines`, de modo que solo nos fijemos en el ambiente y su utilizacion.

### Configuracion
Primero, tenemos los `timesteps`. Esta variable definirá la cantidad de pasos que realizará la red para aprender, mientras más mejor, claro.
Segundo, tenemos la configuración del ambiente. El Objeto de ambiente se instanciará con los siguientes aributos:

- `showDebug` : Entregará información adicional de Debug, como la recompensa en el estado actual, la cantidad de pasos que ha tomado y los máximos, etc.
- `x` : Ancho del problema
- `y` : Alto del Problema

Un ejemplo de instanciación del problema sería: `ContainerYard(showDebug = True)`, en este ejemplo, se mostrará la información de debug y se utilizarán las dimensiones por defecto del problema: 20x5.


### Utilizar el Ambiente
Para utilizar el ambiente se tendra que tener en cuenta las funciones importantes del ambiente. En estas tenemos:

`reset`: este metodo reinicia el problema actual para dar paso al siguiente, retorna una observacion en formato red para alimentar a esta.

`step`: este metodo recibe una accion en formato entero, de 1 a x, donde x sera el ancho del problema. Step deolvera, al igual que reset, una observacion del ambiente. Ademas, devolvera la recompensa de la accion, un boolean que dictamina si es un paso final o no y, opcionalmente, una informacion de debug.

`render`: este metodo mostrara por pantalla el estado actual en la terminal

In [15]:
import gym

#   Se cargan las librerias que se utilizaran para probar el ambiente.  
#   Se utilizara DQN para probar.
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines import DQN

#   Cargamos el Script que contiene el ambiente
from containeryard.containeryard import ContainerYard

#Para evitar que sea mucho tiempo, ejecutaremos 1000 veces, solo para que se vea como funciona.
timesteps = 1000

# Se crea el ambiente:
# showDebug -> Permite ver la informacion de debug que se guardara en la variable info abajo.
# training -> Los generadores crean conjuntos d prueba y entrenamiento, para evitar que este utilice los mismos
#             ejemplos en cada momento. Aqui define que utilizara solo el conjunto de entrenamiento
env = DummyVecEnv([lambda: ContainerYard(showDebug = True)])

#Se crea el modelo, utilizando stable_baselines.
model = DQN(MlpPolicy, env, verbose=1)

#Aqui se entrena!
model.learn(total_timesteps=timesteps)


for i_episode in range(5): # Se probaran 5 problemas diferentes.
    cur_step = 0

    #Comenzamos con un reset, para tomar el siguiente problema.
    print("========= Reset =========")
    print("Problema Numero ",str(i_episode+1))
    print("=========================")

    obs = env.reset()

    while True:
        # El modelo usara la funcion para predecir cual es la mejor
        # accion para la observacion "obs"
        act, _states = model.predict(obs)

        #Se utilizara la accion "act" que entrego la red para ser utilizada en la
        #funcion "step". Esta devolvera una observacion, una recompensa, un boolean que dira
        #si finalizo el episodio y, finalmente informacion de debug en "info".
        obs, reward, done, info = env.step(act)

        #mostrara en la terminal el estado actual.
        env.render()
        cur_step += 1
        print("PASO NUMERO {0}".format(cur_step))
        print("SE ELIGIO EL STACK {0}\n".format(act))

        if done:
            # Si es terminal, entonces se termina y se comienza con el siguiente problema.
            break

print("Se completaron los problemas!")
env.close()








Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 100      |
| mean 100 episode reward | -65.7    |
| steps                   | 318      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 200      |
| mean 100 episode reward | -80      |
| steps                   | 645      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 300      |
| mean 100 episode reward | -80      |
| steps                   | 971      |
--------------------------------------
========= Reset =========
Problema Numero  1
[[ 0.  0.  0.  0.  0.  0.  0.  0.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.